In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

## 01.03. Using a Qu-An Pipeline

In [3]:
from transformers import pipeline

# for understanding we use a small context
context="""
Earth is the third planet from the Sun and the only astronomical object
known to harbor life. While large volumes of water can be found
throughout the Solar System, only Earth sustains liquid surface water.
About 71% of Earth's surface is made up of the ocean, dwarfing
Earth's polar ice, lakes, and rivers. The remaining 29% of Earth's
surface is land, consisting of continents and islands.
Earth's surface layer is formed of several slowly moving tectonic plates,
interacting to produce mountain ranges, volcanoes, and earthquakes.
Earth's liquid outer core generates the magnetic field that shapes Earth's
magnetosphere, deflecting destructive solar winds.
"""

# setting up the pipeline
quan_pipeline = pipeline("question-answering",  # here we are using the question-answering  pipeline
                         model="deepset/minilm-uncased-squad2") # we also choosed the specific model

answer=quan_pipeline(question="How much of earth is land?",
             context=context)
print(answer)

# we got the answer as 29%, also got the start and end character position of the answer
# in addition we got 95% score

{'score': 0.9553403258323669, 'start': 327, 'end': 330, 'answer': '29%'}


In [5]:
# Raising another question
print("\nAnother question :")
print(quan_pipeline( question="How are mountain ranges created?",
             context=context))

# we got the results but the confidence is only 26%


Another question :
{'score': 0.2615407407283783, 'start': 399, 'end': 471, 'answer': "Earth's surface layer is formed of several slowly moving tectonic plates"}


## 01.05 Evaluating Qu-An Performance

In [7]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 976.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [8]:
from evaluate import load
squad_metric = load("squad_v2")

#Ignoring Context & Question as they are not needed for evaluation
#This example is to showcase how the evaluation works based on match between the prediction
#and the correct answer

correct_answer="Paris"

predicted_answers=["Paris",
                 "London",
                 "Paris is one of the best cities in the world"]

cum_predictions=[]
cum_references=[]
for i in range(len(predicted_answers)):

    #Use the input format for predictions
    predictions = [{'prediction_text':predicted_answers[i],
                    'id': str(i),
                    'no_answer_probability': 0.}]
    cum_predictions.append(predictions[0])

    #Use the input format for naswers
    references = [{'answers': {'answer_start': [1],
                               'text': [correct_answer]},
                   'id': str(i)}]
    cum_references.append(references[0])

    results = squad_metric.compute(predictions=predictions,
                                   references=references)
    print("F1 is", results.get('f1'),
          " for answer :", predicted_answers[i])

#Compute for cumulative Results
cum_results=squad_metric.compute(predictions=cum_predictions,
                                 references=cum_references)
print("\n Cumulative Results : \n",cum_results)

F1 is 100.0  for answer : Paris
F1 is 0.0  for answer : London
F1 is 22.22222222222222  for answer : Paris is one of the best cities in the world

 Cumulative Results : 
 {'exact': 33.333333333333336, 'f1': 40.74074074074074, 'total': 3, 'HasAns_exact': 33.333333333333336, 'HasAns_f1': 40.74074074074074, 'HasAns_total': 3, 'best_exact': 33.333333333333336, 'best_exact_thresh': 0.0, 'best_f1': 40.74074074074074, 'best_f1_thresh': 0.0}
